# Notebook to overplot MIRI photometry on existing PROSPECTOR fits

In [ ]:
import os

os.environ["SPS_HOME"] = os.path.expanduser("~/fsps")
os.environ["PYTHONPATH"] = os.environ.get("PYTHONPATH", "") + ":" + os.path.expanduser("~/fsps/python")
os.environ["LD_LIBRARY_PATH"] = os.environ.get("LD_LIBRARY_PATH", "") + ":" + os.path.expanduser("~/fsps/src")
os.environ["PATH"] = os.environ["PATH"] + ":" + os.path.expanduser("~/fsps/bin")

from prosparams_emlinesfit import build_obs, build_model, build_sps
import sedpy
import numpy as np
import matplotlib.pyplot as plt
import prospect.io.read_results as reader

from astropy.io import fits
from astropy import units as u
from prospect.io.read_results import results_from
from astropy.table import Table



Let's load the data from some example output

In [ ]:
catalogue =  '/home/bpc/University/master/Red_Cardinal/catalogues/cat_targets.fits'

with fits.open(catalogue) as catalog_hdul:
        cat_data = catalog_hdul[1].data
        ids = cat_data['id']
        ra = cat_data['ra']
        dec = cat_data['dec']
        zred = cat_data['z_best']

# Combine IDs and redshifts into an array of tuples
output_array = np.array(list(zip(ids, zred)), dtype=[('galid', int), ('zred', '<f8')])

# Save to text file
np.savetxt('/home/bpc/University/master/Red_Cardinal/catalogues/redshifts.txt',
           output_array,
           fmt='%d %.8f',
           header='galid zred')

# Check output files
res, obs, mod = results_from('/home/bpc/University/master/Red_Cardinal/prospector_outputs/output_7102_allfal_v1.4phot_0.3sigma_solar_marg_1733274947_mcmc.h5')

print(res['theta_labels'])       # Parameter names
print(res['chain'].shape)        # Shape of the posterior samples
print(res['bestfit'].keys())     # Best-fit photometry, spectrum, chi2, etc.


Define functions

In [ ]:
def get_zred(objid):
    """Get redshift for given object ID"""
    global id_to_z
    return id_to_z.get(objid, None)

def load_miri_photometry(objid, phot_table):
    """Load MIRI photometry for given object ID"""
    # Update this path to your MIRI photometry file
    miri_phot = Table.read('/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI_v4.fits')
    
    # Filter for your object
    ph = miri_phot[miri_phot['ID'] == str(objid)]
    
    if len(ph) == 0:
        return None
    
    # MIRI filter dictionary - add your specific filters here
    filter_dict_miri = {
        'F770W': 'jwst_f770w',
        #'F1000W': 'jwst_f1000w',
        #'F1280W': 'jwst_f1280w',
        #'F1500W': 'jwst_f1500w',
        'F1800W': 'jwst_f1800w'
        #'F2100W': 'jwst_f2100w',
        #'F2550W': 'jwst_f2550w'
    }
    
    # Load the MIRI filters
    miri_filter_names = list(filter_dict_miri.values())
    miri_filters = sedpy.observate.load_filters(miri_filter_names)
    
    # Extract fluxes and errors
    miri_fluxes = []
    miri_errors = []
    
    flux = ph['Flux'][0] if len(ph) > 0 else np.nan
    flux_err = ph['Flux_Err'][0] if len(ph) > 0 else np.nan
    
    # Loop through flux values
    for obs in flux:
        miri_fluxes.append(obs)
    
    for obs in flux_err:
        miri_errors.append(flux_err)
    
    # Convert to maggies (assuming input is in Jy)
    miri_maggies = np.array(miri_fluxes) / 3631
    miri_maggies_err = np.array(miri_errors) / 3631
    
    # Get effective wavelengths
    miri_wave_eff = np.array([f.wave_effective for f in miri_filters])
    
    return {
        'maggies': miri_maggies,
        'maggies_err': miri_maggies_err,
        'wave_eff': miri_wave_eff,
        'filters': miri_filters,
        'filter_names': list(filter_dict_miri.keys())
    }

def plot_existing_fit_with_miri(objid, phot_table, prospector_output_file):
    """
    Plot existing PROSPECTOR fit and overlay MIRI photometry
    
    Parameters:
        objid : int 
            ID of the galaxy
        prospector_output_file : str 
            Path to existing PROSPECTOR .h5 output file
    """
    
    # Load existing PROSPECTOR results
    results, obs, model = reader.results_from(prospector_output_file)
    
    print(obs)
    # Get redshift
    zred = get_zred(objid)
    
    # Load MIRI photometry
    miri_data = load_miri_photometry(objid, phot_table)
    
    # Get MAP parameters from existing fit
    chains = results['chain']
    log_probabilities = results['lnprobability']
    max_prob_index = np.argmax(log_probabilities)
    map_parameters = chains[max_prob_index]
    
    # Create the plot
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # Plot 1: SED with photometry
    # Convert observed wavelengths to rest-frame
    rest_wave_phot = obs['phot_wave'] / (1 + zred)
    rest_wave_miri = miri_data['wave_eff'] / (1 + zred)
    
    # Plot existing photometry
    ax1.errorbar(rest_wave_phot, obs['maggies'] * 3631, 
                yerr=obs['maggies_unc'] * 3631,
                fmt='o', color='blue', label='HST/NIRCam Photometry', alpha=0.7)
    
    # Plot MIRI photometry
    valid_miri = ~np.isnan(miri_data['maggies'])
    ax1.errorbar(rest_wave_miri[valid_miri], miri_data['maggies'][valid_miri] * 3631,
                yerr=miri_data['maggies_err'][valid_miri] * 3631,
                fmt='s', color='red', label='MIRI Photometry', alpha=0.7, markersize=8)
    
    # If you have the model spectrum from the fit, plot it
    if 'model_spectrum' in results:
        model_wave = results['model_wave'] / (1 + zred)  # Convert to rest-frame
        ax1.plot(model_wave, results['model_spectrum'], 
                color='green', alpha=0.8, label='PROSPECTOR Model')
    
    ax1.set_xlabel('Rest-frame Wavelength (Å)')
    ax1.set_ylabel('Flux (µJy)')
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.legend()
    ax1.set_title(f'Object {objid}: PROSPECTOR Fit + MIRI Photometry (z={zred:.3f})')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Spectrum (if available)
    if obs['spectrum'] is not None:
        rest_wave_spec = obs['wavelength'] / (1 + zred)
        ax2.plot(rest_wave_spec, obs['spectrum'], 
                color='black', alpha=0.5, label='Observed Spectrum')
        ax2.set_xlabel('Rest-frame Wavelength (Å)')
        ax2.set_ylabel('Flux (arbitrary units)')
        ax2.legend()
        ax2.set_title('Spectrum')
        ax2.grid(True, alpha=0.3)
    else:
        ax2.text(0.5, 0.5, 'No spectrum available', 
                transform=ax2.transAxes, ha='center', va='center')
        ax2.set_title('Spectrum: Not Available')
    
    plt.tight_layout()
    plt.savefig(f'prospector_miri_comparison_{objid}.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return fig

def plot_multiple_objects(objid_list, output_dir):
    """
    Plot multiple objects with their PROSPECTOR fits and MIRI data
    
    Parameters:
    objid_list: list - List of object IDs to plot
    output_dir: str - Directory containing PROSPECTOR output files
    """
    
    import os
    
    # Find all PROSPECTOR output files
    output_files = [f for f in os.listdir(output_dir) if f.endswith('.h5')]
    
    for objid in objid_list:
        # Find the corresponding output file
        matching_files = [f for f in output_files if f'_{objid}_' in f]
        
        if matching_files:
            output_file = os.path.join(output_dir, matching_files[0])
            print(f"Plotting object {objid} using file: {matching_files[0]}")
            
            try:
                plot_existing_fit_with_miri(objid, output_file)
            except Exception as e:
                print(f"Error plotting object {objid}: {e}")
        else:
            print(f"No PROSPECTOR output file found for object {objid}")


Example usage:

In [ ]:
# Directory containing your PROSPECTOR outputs
output_dir = "/home/bpc/University/master/Red_Cardinal/prospector_outputs/"

# Example: plot a single object
objid = 16874 #19681  # Replace with your object ID
objid = 16615

# Find the output file for this object
import os
output_files = [f for f in os.listdir(output_dir) if f.endswith('.h5')]
matching_files = [f for f in output_files if f'_{objid}_' in f]

if matching_files:
    output_file = os.path.join(output_dir, matching_files[0])
    phot_table = '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI_v4.fits'
    plot_existing_fit_with_miri(objid, phot_table, output_file)
else:
    print(f"No output file found for object {objid}")

# Or plot multiple objects:
# objid_list = [12345, 67890, 11111]  # Replace with your object IDs
# plot_multiple_objects(objid_list, output_dir)